## Modelo

Neste notebook iremos criar nosso modelo utilizando o framework [Keras](https://pytorch.org)

## Dependências

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pkl
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import normalization
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
def load(name):
    with open(name+'.pkl', 'rb') as f:
        return pkl.load(f)

In [52]:
fold = ['gender_fold_%d_data.txt' % i for i in range(5)]
X = []
Y = []

for f in fold:
    obj = load('objs/'+f)
    for x, y in zip(obj['imagens'], obj['labels']):
        X.append(x)
        Y.append(y)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X, dtype=np.float16), np.array(Y, dtype=np.uint16), test_size=0.15)

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14152, 228, 228, 3), (2498, 228, 228, 3), (14152, 2), (2498, 2))

## Arquitetura

### Modelo

In [4]:
model = Sequential()
# convolutional layers
model.add(Conv2D(filters=96,
                 kernel_size=(7,7),
                 activation='relu',
                 strides=(4,4),
                 padding='valid',
                 input_shape=(228,228,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=256,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=384,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),  padding='same'))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 56, 56, 96)        14208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 384)       885120    
__________

### Hiper-parâmetros

In [61]:
BATCH_SIZE = 50
EPOCHS = 10000
LR = 1e-3
LR_DECAY = 1e-4 / EPOCHS

In [62]:
model.compile(loss=categorical_crossentropy, 
              optimizer=SGD(lr=LR, decay=LR_DECAY, momentum=0.9),
              metrics=['accuracy'])

### Treino

In [63]:
hist = model.fit(x=X_train, y=y_train, validation_split=0.1, batch_size=BATCH_SIZE, epochs=EPOCHS)
model.save('models/model.h5')

print('Train loss:', model.evaluate(X_train, y_train, verbose=0))
print('Test loss:', model.evaluate(X_test, y_test, verbose=0))

Epoch 1/10000
14152/14152 [==============================] - 649s 46ms/step - loss: 0.7089 - acc: 0.6126
Epoch 2/10000
14152/14152 [==============================] - 570s 40ms/step - loss: 0.5838 - acc: 0.6884
Epoch 3/10000
14152/14152 [==============================] - 582s 41ms/step - loss: 0.5507 - acc: 0.7127
Epoch 4/10000
14152/14152 [==============================] - 621s 44ms/step - loss: 0.5151 - acc: 0.7446
Epoch 5/10000
14152/14152 [==============================] - 584s 41ms/step - loss: 0.4845 - acc: 0.7611
Epoch 6/10000
14152/14152 [==============================] - 573s 40ms/step - loss: 0.4518 - acc: 0.7838
Epoch 7/10000
14152/14152 [==============================] - 622s 44ms/step - loss: 0.4298 - acc: 0.7902
Epoch 8/10000
 1700/14152 [==>...........................] - ETA: 9:13 - loss: 0.4333 - acc: 0.7818

KeyboardInterrupt: 